#### Step-By-Step Setup

##### Initial setup
1. Make a new project like `uv init --lib refproj --build-backend=maturin --no-pin-python`
2. Update versions of templated deps in `Cargo.toml`
3. Add numpy rust dep like `cargo add numpy --optional`

##### Enable standalone build for Rust library
4. In `Cargo.toml`,
    * Update `pyo3` dep to be optional
    * Replace autogenerated `dep:numpy` feature with a `python` feature that we will use to signal building the python bindings
    * Under `[lib]` section, add the option to build as a standalone rust library like `crate-type = ["cdylib", "rlib"]`
    * Set the lib name to be the same as the package name like `name = "refproj"`
5. In `pyproject.toml` under the `[tool.maturin]` block,
    * Update maturin rust module name to match the new lib name like `module-name = "refproj.refproj"`
    * Tell maturin to build with the `python` feature enabled like `features = ["python"]`
    * Add `cffi` as a dep (required for building the python bindings locally)
6. Move rust-python bindings to a file called `python.rs` and empty the `lib.rs`
7. In `python.rs`, change the name of the python module to match the project name like `refproj`
8. In `lib.rs`, 
    * Put the inclusion of the python bindings behind a conditional compilation flag

        ```rust
        #[cfg(feature = "python")]
        pub mod python;
        ```
    
    * Use the readme as the module docstring by putting this at the very top.

        ```rust
        #![doc=include_str!("../README.md")]
        ```

        This will have the added benefit that any rust codeblocks in the **readme will be doctested** automatically.
9. In `__init__.py`, update import to use the updated project name instead of the autogenerated one that might have been like `_core`
10. Rename typing stubs `.pyi` file to reflect project name like `refproj.pyi`
11. Test the two builds
    * Python: pick-your-adventure
        * Local editable python install: `uv pip install -e .`
        * Non-editable local install: `uv pip install .`
        * UV distribution/wheels: `uv build`
    * Rust: `cargo build`

##### Upgrade perf
12. In `Cargo.toml`, turn on optimizations for debug builds & fine-tune release builds
    * This is not always helpful, but is usually much better for computing work - compilation is likely cheap compared to run

    ```toml
    [profile.release]
    opt-level = 3
    lto = true
    codegen-units = 1
    overflow-checks = true

    [profile.dev]
    # Nearly full optimizations for debug builds to test for perf regressions
    # when using maturin develop
    opt-level = 3
    codegen-units = 1
    ```

13. Add a compiler config in `.cargo/config.toml`
    * It just needs these lines, which will enable SSE through 4.1, AVX, and FMA for x86 processors
    * Without this, x86 builds will be quite slow due to the compiler accounting for the possibility that your computer might be antique

    ```toml
    [target.'cfg(any(target_arch = "x86", target_arch = "x86_64"))']
    rustflags = ["-Ctarget-feature=+fma"]
    ```

14. Consider using a native build with all available CPU features like

    `RUSTFLAGS="-Ctarget-cpu=native" uv pip install .`

    but don't ship these as wheels, because the user will probably not have your computer.

##### Build the rest of the package

15. Add all of the `tests`, `examples`, `docs`, etc. that you would do for a python-only and rust-only package
16. If publishing, generate the maturin publish workflow in `.github/workflows` like `maturin generate-ci`
    * Add multiple python versions to the build matrix and name files according to python version

In [1]:
from timeit import timeit
import numpy as np

from refproj import nusselt_turbulent_smooth_duct

In [2]:
def nusselt_turbulent_smooth_duct_numpy(Re, Pr, f):
    """
    Nusselt number correlation for fully-developed turbulent flow in smooth ducts, using the
    Gnielinski correlation.
    Length scale is hydraulic diameter.

    Valid region:
        Re: in [3e3, 5e6]
        Pr: in [0.5, inf]
        f: anywhere the source correlation is valid

    Args:
        Re (float): Reynolds number w.r.t. hydraulic diam
        Pr (float): Prandtl number of bulk fluid
        f (float): Darcy friction factor

    Returns:
        float: [] Nusselt number

    References:
        Original Gnielinski paper (in German):
        [1] Gnielinski, V. "Neue Gleichungen für den Wärme- und den Stoffübergang in turbulent durchströmten Rohren und Kanälen."
            Forsch Ing-Wes 41, 8–16 (1975). https://doi.org/10.1007/BF02559682

        English Wikipedia section: https://en.wikipedia.org/wiki/Nusselt_number#Gnielinski_correlation
    """
    num = (f / 8) * (Re - 1000.0) * Pr
    denom = 1 + 12.7 * (f / 8) ** 0.5 * (Pr ** (2.0 / 3.0) - 1)
    Nu = num / denom
    return Nu  # [] Nusselt number

In [ ]:
# Benchmark rust version against python version

n = int(1e7)

re = np.random.uniform(5e3, 5e6, n)
pr = np.random.uniform(0.5, 0.9, n)
f = np.random.uniform(0.02, 0.1, n)
out = np.zeros(n)

tpy = timeit(lambda: nusselt_turbulent_smooth_duct_numpy(re, pr, f), number=300) / 300
trs = timeit(lambda: nusselt_turbulent_smooth_duct(re, pr, f, out), number=300) / 300

print(f"Python: {tpy:.3f} [s], Rust: {trs:.3f} [s], Python / Rust: {tpy / trs:.1f}")
